In [6]:
import pandas as pd
import pickle as pkl
import re
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

import stanza
# stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

def evaluation_metrics(pred, gt):
    print("Prediction: ", len(set(pred)))
    print("GT: ",len(set(gt)))
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    print(TP,FP,FN)
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score 

def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

def get_term_(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for token, label in zip(tokens, labels):
            if label == 'B':
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # Check b_pos = 0 không
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms  

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hanhtran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2021-05-18 09:19:26 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-05-18 09:19:26 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-05-18 09:19:26 INFO: Use device: cpu
2021-05-18 09:19:26 INFO: Loading: tokenize
2021-05-18 09:19:26 INFO: Loading: pos
2021-05-18 09:19:26 INFO: Loading: lemma
2021-05-18 09:19:26 INFO: Done loading processors!


In [20]:
def get_term(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                # Check b_pos = 0 không
                if b_pos != 0:
                    if (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if len(nlp(str(tokens[b_pos - 1])).sentences) > 0:
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            if len(nlp(str(tokens[b_pos])).sentences) > 0:
                                c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                                if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and ((b_word.upos == 'NOUN') or (b_word.upos == 'ADJ')):
                                    terms.append(' '.join([b_word.text] + term))
                    if (tokens[i] != '') and (tokens[i] != ' '):
                        if len(nlp(str(tokens[i])).sentences) > 0:
                            a_word = nlp(str(tokens[i])).sentences[0].words[0]
                            if (a_word.text != 'None') and (a_word.upos == 'NOUN'):
                                terms.append(' '.join(term + [a_word.text]))
                    
                    #ADJ NOUN NOUN
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[i])).sentences) > 0:
                            a_word = nlp(str(tokens[i])).sentences[0].words[0]
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            if len(nlp(str(tokens[b_pos])).sentences) > 0:
                                c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                                # Check vị trí b_pos - 1: terms.append()
                                if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (b_word.text != 'None') and ((b_word.upos == 'ADJ') and (a_word.upos == 'NOUN')):
                                    terms.append(' '.join([b_word.text] + term + [a_word.text]))
#                                     print(terms)
                    #ADJ ADJ NOUN              
                    if (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                            b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                            # Check vị trí b_pos - 1: terms.append()
                            if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'ADJ')):
                                terms.append(' '.join([b1_word.text] +[b_word.text] + term))
#                                 print(terms)
                    #NOUN ADJ NOUN
                    if (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                            b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                            # Check vị trí b_pos - 1: terms.append()
                            if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'NOUN')):
                                terms.append(' '.join([b1_word.text] +[b_word.text] + term))
#                                 print(terms)
                                
                # NOUN NOUN NOUN
                if b_pos != 0 and i + 1 < len(tokens):
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' '):
                        if (len(nlp(str(tokens[i])).sentences) > 0 and len(nlp(str(tokens[i+1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                            a1_word = nlp(str(tokens[i+1])).sentences[0].words[0] 
                            a_word = nlp(str(tokens[i])).sentences[0].words[0]
                            c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                            if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (a_word.text != 'None') and (a1_word.text != 'None') and ((a_word.upos == 'NOUN') and (a1_word.upos == 'NOUN')):
                                terms.append(' '.join( term + [a_word.text] +[a1_word.text]))
#                                 print(terms)
                if b_pos != 0:
                    # RULE 2
                    if (tokens[b_pos-2] != '') and (tokens[b_pos -2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos-2])).sentences) > 0:
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0]
                            # Check vị trí b_pos - 1: terms.append()
                            if  (b_word.text == '-') and (b1_word.text != 'None'):
                                terms.append(' '.join([b1_word.text] + [b_word.text] + term))
                                print(terms)
                    
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # check b_pos - 1
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

In [24]:
groundtruth = pd.read_csv('/Users/hanhtran/Documents/terminology-extraction/ACTER/en/htfl/annotations/htfl_en_terms.ann', sep='	', engine='python',header=None)
gt = list(groundtruth[0]) 

pkl_file = open('../rules/final_preds_sklearn.pkl', 'rb') 
predictions = pkl.load(pkl_file)

In [25]:
preds =  get_term_(predictions)
print(evaluation_metrics(preds, gt))
stop_words = set(stopwords.words('english'))
pred_terms =  set(preds) - set(stop_words)
pred_terms = [x for x in pred_terms if len(x)>1]
pred_terms = [x.lower().strip() for x in pred_terms]
pred_terms = [re.sub(' -','-', x) for x in pred_terms]
pred_terms = [re.sub('- ','-', x) for x in pred_terms]
pred_terms = [re.sub('\(','', x) for x in pred_terms]
pred_terms = [re.sub('\/','', x) for x in pred_terms]
evaluation_metrics(pred_terms, gt)

Prediction:  1674
GT:  2361
701 973 1660
(41.88, 29.69, 34.75)
Prediction:  1656
GT:  2361
704 952 1657


(42.51, 29.82, 35.05)

In [10]:
# preds_lem = lemma(pred_terms)
# gts_lem = lemma(gt)

# print('After lemmatization')
# print(len(set(preds_lem)))
# print(len(set((gts_lem))))
# print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

In [17]:
groundtruth = pd.read_csv('/Users/hanhtran/Documents/terminology-extraction/ACTER/en/htfl/annotations/htfl_en_terms.ann', sep='	', engine='python',header=None)
gt = list(groundtruth[0]) 

pkl_file = open('../rules/final_preds_sklearn.pkl', 'rb') #
predictions = pkl.load(pkl_file)
preds =  get_term_(predictions)
print(evaluation_metrics(preds, gt))
stop_words = set(stopwords.words('english'))
pred_terms =  set(preds) - set(stop_words)
pred_terms = [x for x in pred_terms if len(x)>1]
pred_terms = [x.lower().strip() for x in pred_terms]
pred_terms = [re.sub(' -','-', x) for x in pred_terms]
pred_terms = [re.sub('- ','-', x) for x in pred_terms]
pred_terms = [re.sub('\(','', x) for x in pred_terms]
pred_terms = [re.sub('\/','', x) for x in pred_terms]
evaluation_metrics(pred_terms, gt)

Prediction:  1674
GT:  2361
701 973 1660
(41.88, 29.69, 34.75)
Prediction:  1656
GT:  2361
704 952 1657


(42.51, 29.82, 35.05)